# BIG DATA CS644 FINAL PROJECT 

<ul><li>SENTIMENTAL ANALYSIS</li>
    <li>DAMIAN LOPEZ </li>
<li> New Jersey Insitute of technology</li></ul>
    

### TRAINING DATA

In [56]:
pip install pyspark

     |████████████████████████████████| 212.3 MB 23 kB/s  eta 0:00:01    |███▋                            | 23.9 MB 6.0 MB/s eta 0:00:32     |████▏                           | 27.5 MB 6.0 MB/s eta 0:00:32     |████▊                           | 31.5 MB 9.3 MB/s eta 0:00:20     |████▉                           | 32.0 MB 9.3 MB/s eta 0:00:20     |█████████▍                      | 62.0 MB 9.0 MB/s eta 0:00:17
     |████████████████████████████████| 198 kB 2.7 MB/s eta 0:00:01
  Created wheel for pyspark: filename=pyspark-3.1.1-py2.py3-none-any.whl size=212767604 sha256=f76fd233108c4baa7e1d8436806ac9c5ee4c27d9d00129911706dff667f9f038
  Stored in directory: /Users/damian/Library/Caches/pip/wheels/23/bf/e9/9f3500437422e2ab82246f25a51ee480a44d4efc6c27e50d33
Successfully built pyspark
Note: you may need to restart the kernel to use updated packages.


In [51]:
import pandas as pd 
from pandas import DataFrame
from tweepy.streaming import StreamListener
import numpy as np  
import matplotlib.pyplot as plt  
import seaborn as sns
import os
import csv

In [22]:
# Display all columns and rows
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [23]:
# Read the data in ****** TRAINING DATA **
tweet_trainDF = pd.read_csv('tweeter_train.csv', header=None)
tweet_trainDF.head()

,0,1,2,3,4,5
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [13]:
# Rename columns
tweet_trainDF.rename(columns = {0 : 'Polarity', 1:'tweet_id', 2:'Date', 3:'Query?', 4:'User', 5:'tweet_text'}, inplace=True)
tweet_trainDF.head()

,Polarity,tweet_id,Date,Query?,User,tweet_text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [25]:
tweet_trainDF.shape

(1600000, 6)

### TESTING DATA

In [35]:
# Reading and Processing ***** TESTING DATA *******
import tweepy 
import socket
import sys
import requests
import requests_oauthlib
import json
#import csv

In [36]:
# variables that will be used in OAuth for connecting to Twitter
ACCESS_TOKEN = '1216833846-zgaGaZMMJ2FIzdXtfLWPjBNhmn29EYs26XmxE7h'
ACCESS_SECRET = 'Yi9flb2I9KoviwODOiBHxzGwKI91WMqV9C40WKLmeArrG'
CONSUMER_KEY = 'ojpw4liQ6rOmfaAV0ZyEmoQOx'
CONSUMER_SECRET = 'PkzQRxU5ejErQyiMlxZrnFhuc1fv1b75zE6icHZz58g3twQGeb'
my_auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
my_auth.set_access_token(ACCESS_TOKEN, ACCESS_SECRET)


In [3]:
#function called get_tweets that will call the Twitter API URL and return the response for a stream of tweets
def get_tweets():
    url = 'https://stream.twitter.com/1.1/statuses/filter.json'
    query_data = [('language', 'en'), ('locations', '-130,-20,100,50'),('track','#')]
    query_url = url + '?' + '&'.join([str(t[0]) + '=' + str(t[1]) for t in query_data])
    response = requests.get(query_url, auth=my_auth, stream=True)
    print(query_url, response)
    return response

In [4]:
def send_tweets_to_spark(http_resp, tcp_connection):
    for line in http_resp.iter_lines():
        try:
            full_tweet = json.loads(line)
            tweet_text = full_tweet['text']
            tweet_screenName = full_tweet['user']['screen_name']
            tweet_source = full_tweet['source']
            tweet_created = full_tweet['created_at']
            #print("Tweet Text: " + tweet_text)
            #print ("------------------------------------------")
            tcp_connection.send(tweet_text + '\n')
        except:
            e = sys.exc_info()[0]
            print("Error: %s" % e)

In [34]:
#conn = None
#resp = get_tweets()
#send_tweets_to_spark(resp, conn)
#print(send_tweets_to_spark)

In [40]:
# passing the authorization detalais
api = tweepy.API(my_auth, wait_on_rate_limit=True)
csvFile = open('tweeter_test.csv', 'w')
csvWriter = csv.writer(csvFile)

In [41]:
# Sentimetal Analysis on COVID
search_word = 'COVID'
new_search = search_word+" -filter:retweets"

In [48]:
# Pulling tweets metadata for analysis 
for tweet in tweepy.Cursor(api.search, q=new_search, count=160000, lang='en', since_id=0).items():
    csvWriter.writerow([tweet.id_str, tweet.created_at, tweet.text.encode("utf-8").decode("utf-8"), tweet.user.screen_name, tweet.user.location])
    #outtweets = [[tweet.id_str, tweet.created_at, tweet.text.encode("utf-8").decode("utf-8"), tweet.user.screen_name, tweet.user.location]]
    
# Dataframe
#tweet_testDF = DataFrame(outtweets, columns=['tweet_id','Date', 'tweet_text', 'User', 'User_location'])
#tweet_testDF.to_csv('tweeter_test.csv', index=False)


In [46]:
tweet_testDF = pd.read_csv('tweeter_test.csv', header=None)
tweet_testDF.head()

,0,1,2,3,4
0,1383059528766345216,2021-04-16 14:07:42,"@RawStory Wait, what? I thought that COVID was...",jjgdenisrobert,NaN
1,1383059528695099395,2021-04-16 14:07:42,West Philippine Sea. Racism against Asians sa ...,samuidesu07,NaN
2,1383059527495475205,2021-04-16 14:07:42,Brazil's hospitals running out of sedatives as...,Thesciencevort1,United States
3,1383059526472060929,2021-04-16 14:07:41,"Greetings Tigers🐯 , \nWe have endured rocky tu...",mikayj123,NaN
4,1383059526174318594,2021-04-16 14:07:41,@AP This is misleading. Covid is still out the...,yungperrier,"Vero Beach, FL"


In [47]:
tweet_testDF.shape

(201322, 5)

In [49]:
tweet_testDF.rename(columns={0:'tweet_id', 1:'Date', 2:'tweet_text', 3:'User', 4:'User_location'}, inplace=True)
tweet_testDF.head()

,tweet_id,Date,tweet_text,User,User_location
0,1383059528766345216,2021-04-16 14:07:42,"@RawStory Wait, what? I thought that COVID was...",jjgdenisrobert,NaN
1,1383059528695099395,2021-04-16 14:07:42,West Philippine Sea. Racism against Asians sa ...,samuidesu07,NaN
2,1383059527495475205,2021-04-16 14:07:42,Brazil's hospitals running out of sedatives as...,Thesciencevort1,United States
3,1383059526472060929,2021-04-16 14:07:41,"Greetings Tigers🐯 , \nWe have endured rocky tu...",mikayj123,NaN
4,1383059526174318594,2021-04-16 14:07:41,@AP This is misleading. Covid is still out the...,yungperrier,"Vero Beach, FL"


OSError: [Errno 48] Address already in use

In [61]:
import time
import json
from pyspark import SparkContext
from pyspark.streaming import StreamingContext
 
# Our filter function:
def filter_tweets(tweet):
    json_tweet = json.loads(tweet)
    if json_tweet.has_key('lang'): # When the lang key was not present it caused issues
        if json_tweet['lang'] == 'ar':
            return True # filter() requires a Boolean value
    return False
 
# SparkContext(“local[1]”) would not work with Streaming bc 2 threads are required
sc = SparkContext("local[2]", "Twitter Demo")
ssc = StreamingContext(sc, 10) #10 is the batch interval in seconds
IP = "localhost"
Port = 5555
lines = ssc.socketTextStream(IP, Port)

#lines.foreachRDD( lambda rdd: rdd.filter( filter_tweets ).coalesce(1).saveAsTextFile("./tweets/%f" % time.time()) )
 
# You must start the Spark StreamingContext, and await process termination…
#ssc.start()
ssc.awaitTermination()

ValueError: Cannot run multiple SparkContexts at once; existing SparkContext(app=Twitter Demo, master=local[2]) created by __init__ at <ipython-input-57-35bee233e478>:15 

In [62]:
ssc.stop()

In [58]:
import tweepy
from tweepy import OAuthHandler
from tweepy import Stream
from tweepy.streaming import StreamListener
import socket
import json

In [59]:
ACCESS_TOKEN = '1216833846-zgaGaZMMJ2FIzdXtfLWPjBNhmn29EYs26XmxE7h'
ACCESS_SECRET = 'Yi9flb2I9KoviwODOiBHxzGwKI91WMqV9C40WKLmeArrG'
CONSUMER_KEY = 'ojpw4liQ6rOmfaAV0ZyEmoQOx'
CONSUMER_SECRET = 'PkzQRxU5ejErQyiMlxZrnFhuc1fv1b75zE6icHZz58g3twQGeb'

In [ ]:
class TweetsListener(StreamListener):
  def __init__(self, csocket):
      self.client_socket = csocket
  def on_data(self, data):
    try:
        msg = json.load(data)
        tweet_text = msg['text']
        tweet_screenName = msg['user']['screen_name']
        tweet_source = msg['source']
        tweet_created = msg['created_at']
        self.client_socket(tweet_text, tweet_screenName, tweet_source, tweet_created)
      try:
        msg = json.loads( data )
          
          self.client_socket.send( msg['text'].encode('utf-8') )
          return True
          
      except BaseException as e:
          print("Error on_data: %s" % str(e))
      return True
  def on_error(self, status):
      print(status)
      return True